## Welcome to Ludo Intelligent agent's Notebook 
### Infos about the developer 
#### Badreddine Salah 

In [8]:
board = [0] * 52  # Main track (0 means empty, numbers indicate player tokens)
home_paths = {
    1: [None] * 6,  # Home path for Player 1
    2: [None] * 6,  # Home path for Player 2
    3: [None] * 6,  # Home path for Player 3
    4: [None] * 6   # Home path for Player 4
}
# This array keeps track of player's current position 
safe_zones = {
    1: [-1, -1, -1, -1],  # Player 1's tokens before entering the board
    2: [-1, -1, -1, -1],  # Player 2 //
    3: [-1, -1, -1, -1],  # Player 3 //
    4: [-1, -1, -1, -1]   # Player 4 //
}

In [9]:
def move_token(player_id, token_index, dice_roll):
    position = safe_zones[player_id][token_index]  # Get token's current position

    # If token is in the safe zone and rolls a 6, move it to the start position
    if position == -1 and dice_roll == 6:
        start_positions = {1: 0, 2: 13, 3: 26, 4: 39}
        new_position = start_positions[player_id]
        safe_zones[player_id][token_index] = new_position
        board[new_position] = player_id
        return

    # If token is on the board, calculate new position
    if position >= 0:
        new_position = (position + dice_roll) % 52

        # Check for captures
        if board[new_position] not in (0, player_id):
            opponent = board[new_position]
            for i in range(4):  # Find the opponent's token and send it back
                if safe_zones[opponent][i] == new_position:
                    safe_zones[opponent][i] = -1  # Send back to start

        # Move token
        board[position] = 0  # Clear old position
        board[new_position] = player_id
        safe_zones[player_id][token_index] = new_position

In [10]:
def check_win(player_id):
    return all(pos == "home" for pos in home_paths[player_id])

In [11]:
def agent_move(player_id, dice_roll):
    # Rule 1: If a token is in the safe zone and can enter the board, do that
    for i in range(4):
        if safe_zones[player_id][i] == -1 and dice_roll == 6:
            move_token(player_id, i, dice_roll)
            return

    # Rule 2: Try to capture an opponent if possible
    for i in range(4):
        current_pos = safe_zones[player_id][i]
        if current_pos >= 0:
            new_pos = (current_pos + dice_roll) % 52
            if board[new_pos] not in (0, player_id):  # Opponent is present there ! a chance to kill hehe
                move_token(player_id, i, dice_roll)
                return
                
    # Rule 3: Avoid getting ctached by other players -> choose the safest option if available 
    for i in range(4):
        current_pos = safe_zones[player_id][i]
        if current_pos >= 0:
            new_pos = (current_pos + dice_roll)%52
            test = True
            for j in range(1, 6):
                if board[(new_pos-j)%52] not in (0, player_id):
                    test = False
            if (test):
                move_token(player_id, i, dice_roll)

    # Rule 4: Using probabilities, let's check how closely a player can catch you !
    all_players = [1, 2, 3, 4]
    oponents = [p for p in all_players if p != player_id];
    for i in range(4):
        current_pos = safe_zones[player_id][i]
        if current_pos >= 0:
            probabilities = [0] * 3
            new_pos = (current_pos + dice_roll) % 52
            index = 0
            for j in oponents:
                for k in range(1, 6):
                    if board[(new_pos-k)%52] == j:
                        probabilities[index] += 1/6
                index += 1
            test = True
            for t in range(3):
               if probabilities[t] > 1/6: # there is an oponent with a high chance to kill you, so avoid !
                   test = False
            if test:
                move_token(player_id, i, dice_roll)
            
    
                
    # Rule 5: Move the farthest token forward -> Worst Case !!!
    farthest_token = max(safe_zones[player_id])
    if farthest_token >= 0:
        index = safe_zones[player_id].index(farthest_token)
        move_token(player_id, index, dice_roll)

In [ ]:
import random
import time

def print_board():
    board_visual = [str(x) if x != 0 else '.' for x in board]
    print("\nBoard State:")
    print(" ".join(board_visual))
    print("\nSafe Zones:")
    for player, tokens in safe_zones.items():
        print(f"Player {player}: {tokens}")

def test_game():
    players = [1, 2, 3, 4]
    while True:
        for player in players:
            input(f"Player {player}'s turn. Press Enter to roll the dice...")
            dice_roll = random.randint(1, 6)
            print(f"Player {player} rolled a {dice_roll}")
            agent_move(player, dice_roll)
            print_board()
            time.sleep(1)
            
            if check_win(player):
                print(f"Player {player} wins the game!")
                return

if __name__ == "__main__":
    print("Starting the game...\n")
    test_game()

Starting the game...



Player 1's turn. Press Enter to roll the dice... 


Player 1 rolled a 6

Board State:
1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Safe Zones:
Player 1: [0, -1, -1, -1]
Player 2: [-1, -1, -1, -1]
Player 3: [-1, -1, -1, -1]
Player 4: [-1, -1, -1, -1]


Player 2's turn. Press Enter to roll the dice... 


Player 2 rolled a 4

Board State:
1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Safe Zones:
Player 1: [0, -1, -1, -1]
Player 2: [-1, -1, -1, -1]
Player 3: [-1, -1, -1, -1]
Player 4: [-1, -1, -1, -1]


...

Player 3 rolled a 2

Board State:
1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Safe Zones:
Player 1: [0, -1, -1, -1]
Player 2: [-1, -1, -1, -1]
Player 3: [-1, -1, -1, -1]
Player 4: [-1, -1, -1, -1]


Player 4's turn. Press Enter to roll the dice... 


Player 4 rolled a 3

Board State:
1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Safe Zones:
Player 1: [0, -1, -1, -1]
Player 2: [-1, -1, -1, -1]
Player 3: [-1, -1, -1, -1]
Player 4: [-1, -1, -1, -1]


Player 1's turn. Press Enter to roll the dice... 


Player 1 rolled a 6

Board State:
1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Safe Zones:
Player 1: [0, 0, -1, -1]
Player 2: [-1, -1, -1, -1]
Player 3: [-1, -1, -1, -1]
Player 4: [-1, -1, -1, -1]
